In [26]:
# take natural language as input and convert into sql statements to get data from in memory database 
# use pandas for this activity 
import os 
import openai 
import pandas as pd 
import zipfile
import urllib
import urllib.request

# store the api key as environment variable
openai.api_key = os.getenv("OPENAI_API_KEY")

In [27]:
DOWNLOAD_ROOT="https://www.kaggle.com/api/v1/datasets/download/kyanyoga/sample-sales-data"
DATA_LOCATION="/Users/rohitabhishek/Python_Workspace"
DATA_PATH=os.path.join(DATA_LOCATION, "openai_programs", "udemy_openai", "DATA")

In [ ]:
def fetch_sales_data(data_url=DOWNLOAD_ROOT, data_path=DATA_PATH):
    os.makedirs(data_path, exist_ok=True)

    # give filename 
    zip_file = os.path.join(data_path, "sample-sales-data.zip")

    urllib.request.urlretrieve(data_url, zip_file)

    # extract the data 
    sales_tgz=zipfile.ZipFile(zip_file)
    sales_tgz.extractall(path=data_path)
    sales_tgz.close()

files = os.listdir(DATA_PATH)
if "sales_data_sample.csv" not in files: 
    fetch_sales_data()

In [31]:
os.listdir(DATA_PATH)

['sample-sales-data.zip', 'sales_data_sample.csv']

In [39]:
data_location=os.path.join(DATA_PATH, "sales_data_sample.csv")
df=pd.read_csv(data_location)
df.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


In [40]:
df["YEAR_ID"].value_counts()

YEAR_ID
2004    1345
2003    1000
2005     478
Name: count, dtype: int64

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2823 entries, 0 to 2822
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ORDERNUMBER       2823 non-null   int64  
 1   QUANTITYORDERED   2823 non-null   int64  
 2   PRICEEACH         2823 non-null   float64
 3   ORDERLINENUMBER   2823 non-null   int64  
 4   SALES             2823 non-null   float64
 5   ORDERDATE         2823 non-null   object 
 6   STATUS            2823 non-null   object 
 7   QTR_ID            2823 non-null   int64  
 8   MONTH_ID          2823 non-null   int64  
 9   YEAR_ID           2823 non-null   int64  
 10  PRODUCTLINE       2823 non-null   object 
 11  MSRP              2823 non-null   int64  
 12  PRODUCTCODE       2823 non-null   object 
 13  CUSTOMERNAME      2823 non-null   object 
 14  PHONE             2823 non-null   object 
 15  ADDRESSLINE1      2823 non-null   object 
 16  ADDRESSLINE2      302 non-null    object 


In [42]:
df.groupby("QTR_ID").sum()["SALES"]

QTR_ID
1    2350817.73
2    2048120.30
3    1758910.81
4    3874780.01
Name: SALES, dtype: float64

``` 
We want to give question like what is the total sales per quater which is equivalent to 
SELECT SUM(SALES) FROM TABLE GROUP BY QTR_ID
```

In [43]:
import sqlalchemy
from sqlalchemy import create_engine, text 

In [44]:
# create in-memory database 
temp_db = create_engine("sqlite:///:memory:", echo=True)
data=df.to_sql(name="Sales", con=temp_db)

2025-06-03 18:56:13,012 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-03 18:56:13,015 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Sales")
2025-06-03 18:56:13,015 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-06-03 18:56:13,016 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Sales")
2025-06-03 18:56:13,016 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-06-03 18:56:13,017 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Sales" (
	"index" BIGINT, 
	"ORDERNUMBER" BIGINT, 
	"QUANTITYORDERED" BIGINT, 
	"PRICEEACH" FLOAT, 
	"ORDERLINENUMBER" BIGINT, 
	"SALES" FLOAT, 
	"ORDERDATE" TEXT, 
	"STATUS" TEXT, 
	"QTR_ID" BIGINT, 
	"MONTH_ID" BIGINT, 
	"YEAR_ID" BIGINT, 
	"PRODUCTLINE" TEXT, 
	"MSRP" BIGINT, 
	"PRODUCTCODE" TEXT, 
	"CUSTOMERNAME" TEXT, 
	"PHONE" TEXT, 
	"ADDRESSLINE1" TEXT, 
	"ADDRESSLINE2" TEXT, 
	"CITY" TEXT, 
	"STATE" TEXT, 
	"POSTALCODE" TEXT, 
	"COUNTRY" TEXT, 
	"TERRITORY" TEXT, 
	"CONTACTLASTNAME" TEXT, 
	"CONTACTFIRSTNAME" TEXT, 
	"DEALSIZE" TEXT
)


20

In [45]:
with temp_db.connect() as conn:
    result=conn.execute(text("SELECT QTR_ID, SUM(SALES) from Sales Group by QTR_ID"))

result.all()

2025-06-03 18:56:14,850 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-03 18:56:14,851 INFO sqlalchemy.engine.Engine SELECT QTR_ID, SUM(SALES) from Sales Group by QTR_ID
2025-06-03 18:56:14,851 INFO sqlalchemy.engine.Engine [generated in 0.00096s] ()
2025-06-03 18:56:14,852 INFO sqlalchemy.engine.Engine ROLLBACK


[(1, 2350817.73), (2, 2048120.3), (3, 1758910.81), (4, 3874780.01)]

``` 
Create OPENAI prompt
```

In [46]:
# get details from csv for open ai to understand 
csv_details_prompt = f"""I have a CSV File with following columns: \n
{','.join(str(col) for col in df.columns)}.\nCreate a python function to insert record from this CSV to SQLLite table. 
"""
print(csv_details_prompt)

I have a CSV File with following columns: 

ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE.
Create a python function to insert record from this CSV to SQLLite table. 



In [ ]:
class CreateBot:
    def __init__(self, system_prompt):
        self.system_prompt=system_prompt
        self.message=[{"role" : "system", "content" : system_prompt}]

    def create_question_prompt(self):
        question = ""
        print("To Terminate type END")

        while question!= "END":
            question = input("Your question is: ")
            print ("\n")

            self.message.append({"role": "system", "content": question})

            response=openai.ChatCompletion.create(
                model="gpt-3.5-turbo", 
                messages=self.message
            )

            content=response["choices"][0]["message"]["content"]
            print("\n")
            print(content)
            print("\n")

            self.message.append({"role" : "assistant", "content": content})

In [52]:
python_expert = CreateBot(system_prompt="You are a Python Programming expert.")
python_expert.chat(csv_details_prompt)

TypeError: CreateBot.chat() takes 1 positional argument but 2 were given